In [1]:
import pandas as pd
import re
from mlxtend.frequent_patterns import apriori, association_rules
data = pd.read_csv("Team_Sports_Survey_Preprocessed.csv")
# remove the eg. for each items
def remove_parentheses(text):
    if isinstance(text, str):  
        return re.sub(r"\s*\(.*?\)", "", text)
    return text  

In [2]:
# Get the rows which preferred swimming pool activity which is relaxing in pool
# Split the values into a single items in the "Preferred Swimming Pool Activities"
data["Preferred Swimming Pool Activities"] = data["Preferred Swimming Pool Activities"].astype(str).apply(remove_parentheses).str.split(", ")
# Get only rows where Relaxing in Pool is in the dataset
pool_act = data[data["Preferred Swimming Pool Activities"].apply(lambda x: "Relaxing in Pool" in x)]

In [3]:
# Split the values into a single items
pool_act["Preferred Spa and Wellness Activities"] =pool_act["Preferred Spa and Wellness Activities"].apply(remove_parentheses).str.split(",")  

# Convert the items into a transactional data 
transactions = pool_act["Preferred Spa and Wellness Activities"].explode().str.strip().reset_index() 
transactions_encoded = transactions.pivot_table(index="index", columns="Preferred Spa and Wellness Activities", aggfunc=lambda x: 1, fill_value=0) 

C:\Users\joaqu\AppData\Local\Temp\ipykernel_2260\2748023349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pool_act["Preferred Spa and Wellness Activities"] =pool_act["Preferred Spa and Wellness Activities"].apply(remove_parentheses).str.split(",")


In [4]:
# Use Apriori algorithm to find frequent association
frequent_items= apriori(transactions_encoded, min_support=0.3, use_colnames=True)# threshold 30%
rule = association_rules(frequent_items)
rule[['antecedents','consequents','support', 'confidence', 'lift']].sort_values("confidence", ascending=False)

C:\Users\joaqu\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift
2,"(Sauna and Steam Rooms, Massage Therapy)",(Spa Treatment),0.418182,0.884615,1.247535
3,"(Sauna and Steam Rooms, Spa Treatment)",(Massage Therapy),0.418182,0.884615,1.186679
1,(Spa Treatment),(Massage Therapy),0.618182,0.871795,1.169481
0,(Massage Therapy),(Spa Treatment),0.618182,0.829268,1.169481
